# **Aim of the notebook:**
To check if the bias exsists in the Training dataset of IndicTrans2. Bias here means if the translation of ambiguos words is done more for paternal than maternal. 
To check the ratio of sentences in paternal vs maternal in both training and dataset.

## Setup

In [1]:
%%capture
!git clone https://github.com/AI4Bharat/IndicTrans2.git

In [2]:
%%capture
%cd /content/IndicTrans2/huggingface_interface

In [3]:
%%capture
!python3 -m pip install nltk sacremoses pandas regex mock transformers>=4.33.2 mosestokenizer


In [4]:
!python3 -c "import nltk; nltk.download('punkt')"


[nltk_data] Downloading package punkt to /home/sofia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
!python3 -m pip install bitsandbytes scipy accelerate datasets


In [6]:
!python3 -m pip install sentencepiece


In [7]:
print("Installed not all the required packages")

print("Installed all the required packages")

!git clone https://github.com/VarunGumma/IndicTransToolkit
%cd IndicTransToolkit
!python3 -m pip install --editable ./
%cd ..

Installed not all the required packages
Installed all the required packages


fatal: destination path 'IndicTransToolkit' already exists and is not an empty directory.
/home/sofia/multilingual/IndicTransToolkit
Obtaining file:///home/sofia/multilingual/IndicTransToolkit
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/VarunGumma/indic_nlp_library to /tmp/pip-install-e0ecyfqw/indic-nlp-library-it2_6fb74f4f80a64fddabe9e6bc879df959
  Running command git clone --filter=blob:none --quiet https://github.com/VarunGumma/indic_nlp_library /tmp/pip-install-e0ecyfqw/indic-nlp-library-it2_6fb74f4f80a64fddabe9e6bc879df959
  Resolved https://github.com/VarunGumma/indic_nlp_library to commit 601521e05ed0ed8f2165ac317a47d186e25b6f0d
  Preparing metadata (setup.py) ... done
  Attempting uninstall: IndicTransToolkit
    Found existing installation: IndicTransToolkit 1.0.2
    Uninstalling IndicTransToolkit-1.0.2:
      Successfully uninstalled IndicTransToolkit-1.0.2
  DEPRECATION: Legacy editable install of IndicTransToolkit==1.0.2 from file:///home/sofia/mu

In [2]:
import pandas as pd

Restart after indic download

# Languages Involved:


In [12]:
languages = [ 'or', 'hi', 'ta', 'te','bn', 'gu',  'kn', 'ml', 'mr',  'pa'  ]

# Datasets involved:

In [12]:
# path = 'https://indictrans2-public.objectstore.e2enetworks.net/IN22_testset.zip'
# # path = 'https://indictrans2-public.objectstore.e2enetworks.net/flores-22_dev.zip'
# zip_path = path.split('/')[-1]

# # Download the dataset using above link
# !wget $path -O $zip_path


In [13]:
# !unzip $zip_path

Archive:  IN22_testset.zip
   creating: IN22_testset/
   creating: IN22_testset/conv/
  inflating: IN22_testset/conv/test.npi_Deva  
  inflating: IN22_testset/conv/test.snd_Deva  
  inflating: IN22_testset/conv/test.ory_Orya  
  inflating: IN22_testset/conv/test.sat_Olck  
  inflating: IN22_testset/conv/metadata.tsv  
  inflating: IN22_testset/conv/test.brx_Deva  
  inflating: IN22_testset/conv/test.urd_Arab  
  inflating: IN22_testset/conv/test.tam_Taml  
  inflating: IN22_testset/conv/test.tel_Telu  
  inflating: IN22_testset/conv/test.eng_Latn  
  inflating: IN22_testset/conv/test.mni_Mtei  
  inflating: IN22_testset/conv/test.guj_Gujr  
  inflating: IN22_testset/conv/test.ben_Beng  
  inflating: IN22_testset/conv/test.gom_Deva  
  inflating: IN22_testset/conv/test.asm_Beng  
  inflating: IN22_testset/conv/test.pan_Guru  
  inflating: IN22_testset/conv/test.mar_Deva  
  inflating: IN22_testset/conv/test.mai_Deva  
  inflating: IN22_testset/conv/test.mal_Mlym  
  inflating: IN22_test

## Langs and scripts

In [15]:
# create a list with data (just include within bracket): Assamese (asm_Beng)	Kashmiri (Arabic) (kas_Arab)	Punjabi (pan_Guru)
# Bengali (ben_Beng)	Kashmiri (Devanagari) (kas_Deva)	Sanskrit (san_Deva)
# Bodo (brx_Deva)	Maithili (mai_Deva)	Santali (sat_Olck)
# Dogri (doi_Deva)	Malayalam (mal_Mlym)	Sindhi (Arabic) (snd_Arab)
# English (eng_Latn)	Marathi (mar_Deva)	Sindhi (Devanagari) (snd_Deva)
# Konkani (gom_Deva)	Manipuri (Bengali) (mni_Beng)	Tamil (tam_Taml)
# Gujarati (guj_Gujr)	Manipuri (Meitei) (mni_Mtei)	Telugu (tel_Telu)
# Hindi (hin_Deva)	Nepali (npi_Deva)	Urdu (urd_Arab)
# Kannada (kan_Knda)	Odia (ory_Orya)	

lang_script_list = [
    # 'asm_Beng', 'kas_Arab',
                     'pan_Guru', 'ben_Beng', 
                    #  'kas_Deva', 'san_Deva', 'brx_Deva', 'mai_Deva', 'sat_Olck', 'doi_Deva',
                       'mal_Mlym',
                     #    #  'snd_Arab',
                     #    #    'eng_Latn', 
                           'mar_Deva', 
                     #    #    'snd_Deva', 'gom_Deva', 'mni_Beng', 
                           'tam_Taml', 'guj_Gujr', 
                     #    #    'mni_Mtei', 
                           'tel_Telu', 'hin_Deva', 
                     #    #    'npi_Deva', 'urd_Arab', 
                           'kan_Knda', 
                           'ory_Orya'
                           ]



In [16]:
# Traverse through each folder of XXXU and print the folder names
import os
import pandas as pd
import tqdm

# Create a list to store the folder names
# folders = []
 # create a list of 10 items of dataframes in an array
df_list = {
    i : pd.DataFrame() for i in (lang_script_list)
}

df_list

{'pan_Guru': Empty DataFrame
 Columns: []
 Index: [],
 'ben_Beng': Empty DataFrame
 Columns: []
 Index: [],
 'mal_Mlym': Empty DataFrame
 Columns: []
 Index: [],
 'mar_Deva': Empty DataFrame
 Columns: []
 Index: [],
 'tam_Taml': Empty DataFrame
 Columns: []
 Index: [],
 'guj_Gujr': Empty DataFrame
 Columns: []
 Index: [],
 'tel_Telu': Empty DataFrame
 Columns: []
 Index: [],
 'hin_Deva': Empty DataFrame
 Columns: []
 Index: [],
 'kan_Knda': Empty DataFrame
 Columns: []
 Index: [],
 'ory_Orya': Empty DataFrame
 Columns: []
 Index: []}

In [19]:

dataset_root = 'IN22_testset/'
# Traverse through each folder of XXXU and print the folder names
for folder in os.listdir(dataset_root):
        for indic_lang in lang_script_list:
            # print the file names within the sub folders
            file = 'test.'+indic_lang

            df = pd.DataFrame(columns=['eng_Latn', indic_lang])
            with open(dataset_root+folder+'/'+'test.eng_Latn') as f:
                            df_en = f.readlines()
            with open(dataset_root+folder+'/'+file) as f:
                            df_ind = f.readlines()
            df = pd.DataFrame()
            df['eng_Latn'] = df_en
            df[indic_lang] = df_ind
            
            df_list[indic_lang] = pd.concat([df_list[indic_lang], df], axis=0)

                    
df_list        



# BPCC/nllb_filtered/eng_Latn-ory_Orya/train.eng_Latn

{'pan_Guru':                                                eng_Latn  \
 0                 Mom, let's go for a movie tomorrow.\n   
 1                       I don't have to go to school.\n   
 2                                    It is a holiday.\n   
 3            Oh, tomorrow is the 14th of April right?\n   
 4      Your dad will also have the day off from work.\n   
 ...                                                 ...   
 1019  The rare and threatened animal species identif...   
 1020  The state has several beaches in its coastal d...   
 1021  Garuda Indonesia conducts weekly open-charter ...   
 1022  As of December 2020, 553 wildlife sanctuaries ...   
 1023  In Karaikal liquor is cheaper than in the neig...   
 
                                                pan_Guru  
 0                ਮਾਂ , ਚਲੋ ਕੱਲ੍ਹ ਫ਼ਿਲਮ ਦੇਖਣ ਚਲਦੇ ਹਾਂ |\n  
 1                          ਮੈਂ ਕੱਲ੍ਹ ਸਕੂਲ ਨਹੀਂ ਜਾਣਾ |\n  
 2                                           ਛੁੱਟੀ ਹੈ|\n  
 3                   ਅੱਛਾ, ਕੱਲ

In [28]:
# ambiguos relations:
ambiguos = [
    # 'uncle', 'aunt', 'grandfather', 'grandmother',
    #          'brother in law', 'brother-in-law', 'sister in law', 'sister-in-law',
             'cousin', 'nephew', 'niece', 'child', 'children', 
             # singular possessive
             'uncle\'s', 'aunt\'s', 'grandfather\'s', 'grandmother\'s', 
                'brother in law\'s', 'brother-in-law\'s', 'sister in law\'s', 'sister-in-law\'s',
                'cousin\'s', 'nephew\'s', 'niece\'s', 'child\'s', 'children\'s',
                # plural possessive
                'uncles\'', 'aunts\'', 'grandfathers\'', 'grandmothers\'',
                'brothers in law\'s', 'brothers-in-law\'s', 'sisters in law\'s', 'sisters-in-law\'s',
                'cousins\'', 'nephews\'', 'nieces\'', 
                # plural
                'uncles', 'aunts', 'grandfathers', 'grandmothers',
                'brothers in law', 'brothers-in-law', 'sisters in law', 'sisters-in-law',
                'cousins', 'nephews', 'nieces',

             ]






In [31]:
'test_csv__/'+dataset_root[:-1]+'_'+lang+'.csv'

'test_csv__/IN22_testset_ory_Orya.csv'

In [32]:
# remove duplicates

# do for all languages in df_list
for lang in df_list:
    df_list[lang] = df_list[lang][~df_list[lang].index.duplicated()]
    print(lang, df_list[lang].shape)
    # save the dataframes
    df_list[lang].to_csv('test_csv__/'+dataset_root[:-1]+'_'+lang+'.csv', index=False)
    

pan_Guru (1503, 2)
ben_Beng (1503, 2)
mal_Mlym (1503, 2)
mar_Deva (1503, 2)
tam_Taml (1503, 2)
guj_Gujr (1503, 2)
tel_Telu (1503, 2)
hin_Deva (1503, 2)
kan_Knda (1503, 2)
ory_Orya (1503, 2)


In [25]:
df_list_copy = df_list.copy() # make a copy of the original dataframes to save them for later use

## Filtering Dataframes as per English Ambiguous Relations

In [35]:
# tqdm progress bar for the for loop on df_list
# import tqdm
df_dict = {} # this dictionary will have key as language from df_list and value as another dictionary.
# this inner dictionary will have key as the word from ambiguos and value as the filtered dataframe containing that word in the english column. 

# using all ambiguous words make a regex pattern and filter the dataframes
# do for all languages in df_list

regex_pattern = '|'.join(ambiguos)

for lang in tqdm.tqdm(df_list):
    df = df_list[lang]
    df_dict[lang] = {}
    for word in ambiguos:
        df_dict[lang][word] = df[df['eng_Latn'].str.contains(word, case=False, )]
        print(lang, word, df_dict[lang][word].shape)
        df_dict[lang][word].to_csv(f'test_csv__/{lang}/{lang}_{word}_{dataset_root[:-1]}.csv', index=False)
    
    # .to_csv(f'{lang}_bpcc.csv', index=False)

df_dict

 20%|██        | 2/10 [00:00<00:00, 11.59it/s]

pan_Guru cousin (1, 2)
pan_Guru nephew (0, 2)
pan_Guru niece (0, 2)
pan_Guru child (10, 2)
pan_Guru children (8, 2)
pan_Guru uncle's (0, 2)
pan_Guru aunt's (0, 2)
pan_Guru grandfather's (0, 2)
pan_Guru grandmother's (0, 2)
pan_Guru brother in law's (0, 2)
pan_Guru brother-in-law's (0, 2)
pan_Guru sister in law's (0, 2)
pan_Guru sister-in-law's (0, 2)
pan_Guru cousin's (0, 2)
pan_Guru nephew's (0, 2)
pan_Guru niece's (0, 2)
pan_Guru child's (1, 2)
pan_Guru children's (2, 2)
pan_Guru uncles' (0, 2)
pan_Guru aunts' (0, 2)
pan_Guru grandfathers' (0, 2)
pan_Guru grandmothers' (0, 2)
pan_Guru brothers in law's (0, 2)
pan_Guru brothers-in-law's (0, 2)
pan_Guru sisters in law's (0, 2)
pan_Guru sisters-in-law's (0, 2)
pan_Guru cousins' (0, 2)
pan_Guru nephews' (0, 2)
pan_Guru nieces' (0, 2)
pan_Guru uncles (0, 2)
pan_Guru aunts (0, 2)
pan_Guru grandfathers (0, 2)
pan_Guru grandmothers (0, 2)
pan_Guru brothers in law (0, 2)
pan_Guru brothers-in-law (0, 2)
pan_Guru sisters in law (0, 2)
pan_Guru 

 40%|████      | 4/10 [00:00<00:00, 12.60it/s]

mal_Mlym children's (2, 2)
mal_Mlym uncles' (0, 2)
mal_Mlym aunts' (0, 2)
mal_Mlym grandfathers' (0, 2)
mal_Mlym grandmothers' (0, 2)
mal_Mlym brothers in law's (0, 2)
mal_Mlym brothers-in-law's (0, 2)
mal_Mlym sisters in law's (0, 2)
mal_Mlym sisters-in-law's (0, 2)
mal_Mlym cousins' (0, 2)
mal_Mlym nephews' (0, 2)
mal_Mlym nieces' (0, 2)
mal_Mlym uncles (0, 2)
mal_Mlym aunts (0, 2)
mal_Mlym grandfathers (0, 2)
mal_Mlym grandmothers (0, 2)
mal_Mlym brothers in law (0, 2)
mal_Mlym brothers-in-law (0, 2)
mal_Mlym sisters in law (0, 2)
mal_Mlym sisters-in-law (0, 2)
mal_Mlym cousins (0, 2)
mal_Mlym nephews (0, 2)
mal_Mlym nieces (0, 2)
mar_Deva cousin (1, 2)
mar_Deva nephew (0, 2)
mar_Deva niece (0, 2)
mar_Deva child (10, 2)
mar_Deva children (8, 2)
mar_Deva uncle's (0, 2)
mar_Deva aunt's (0, 2)
mar_Deva grandfather's (0, 2)
mar_Deva grandmother's (0, 2)
mar_Deva brother in law's (0, 2)
mar_Deva brother-in-law's (0, 2)
mar_Deva sister in law's (0, 2)
mar_Deva sister-in-law's (0, 2)
mar_D

 60%|██████    | 6/10 [00:00<00:00, 13.02it/s]

guj_Gujr brother in law's (0, 2)
guj_Gujr brother-in-law's (0, 2)
guj_Gujr sister in law's (0, 2)
guj_Gujr sister-in-law's (0, 2)
guj_Gujr cousin's (0, 2)
guj_Gujr nephew's (0, 2)
guj_Gujr niece's (0, 2)
guj_Gujr child's (1, 2)
guj_Gujr children's (2, 2)
guj_Gujr uncles' (0, 2)
guj_Gujr aunts' (0, 2)
guj_Gujr grandfathers' (0, 2)
guj_Gujr grandmothers' (0, 2)
guj_Gujr brothers in law's (0, 2)
guj_Gujr brothers-in-law's (0, 2)
guj_Gujr sisters in law's (0, 2)
guj_Gujr sisters-in-law's (0, 2)
guj_Gujr cousins' (0, 2)
guj_Gujr nephews' (0, 2)
guj_Gujr nieces' (0, 2)
guj_Gujr uncles (0, 2)
guj_Gujr aunts (0, 2)
guj_Gujr grandfathers (0, 2)
guj_Gujr grandmothers (0, 2)
guj_Gujr brothers in law (0, 2)
guj_Gujr brothers-in-law (0, 2)
guj_Gujr sisters in law (0, 2)
guj_Gujr sisters-in-law (0, 2)
guj_Gujr cousins (0, 2)
guj_Gujr nephews (0, 2)
guj_Gujr nieces (0, 2)
tel_Telu cousin (1, 2)
tel_Telu nephew (0, 2)
tel_Telu niece (0, 2)
tel_Telu child (10, 2)
tel_Telu children (8, 2)
tel_Telu uncle

100%|██████████| 10/10 [00:00<00:00, 12.99it/s]

hin_Deva nieces (0, 2)
kan_Knda cousin (1, 2)
kan_Knda nephew (0, 2)
kan_Knda niece (0, 2)
kan_Knda child (10, 2)
kan_Knda children (8, 2)
kan_Knda uncle's (0, 2)
kan_Knda aunt's (0, 2)
kan_Knda grandfather's (0, 2)
kan_Knda grandmother's (0, 2)
kan_Knda brother in law's (0, 2)
kan_Knda brother-in-law's (0, 2)
kan_Knda sister in law's (0, 2)
kan_Knda sister-in-law's (0, 2)
kan_Knda cousin's (0, 2)
kan_Knda nephew's (0, 2)
kan_Knda niece's (0, 2)
kan_Knda child's (1, 2)
kan_Knda children's (2, 2)
kan_Knda uncles' (0, 2)
kan_Knda aunts' (0, 2)
kan_Knda grandfathers' (0, 2)
kan_Knda grandmothers' (0, 2)
kan_Knda brothers in law's (0, 2)
kan_Knda brothers-in-law's (0, 2)
kan_Knda sisters in law's (0, 2)
kan_Knda sisters-in-law's (0, 2)
kan_Knda cousins' (0, 2)
kan_Knda nephews' (0, 2)
kan_Knda nieces' (0, 2)
kan_Knda uncles (0, 2)
kan_Knda aunts (0, 2)
kan_Knda grandfathers (0, 2)
kan_Knda grandmothers (0, 2)
kan_Knda brothers in law (0, 2)
kan_Knda brothers-in-law (0, 2)
kan_Knda sisters 

{'pan_Guru': {'cousin':                                               eng_Latn  \
  443  Everything is good here except my cousin broth...   
  
                                                pan_Guru  
  443  ਇੱਥੇ ਸਭ ਕੁੱਝ ਠੀਕ ਹੈ ਮੇਰੇ ਚਾਚੇ ਦੇ ਮੁੰਡੇ ਨੂੰ ਛੱਡ...  ,
  'nephew': Empty DataFrame
  Columns: [eng_Latn, pan_Guru]
  Index: [],
  'niece': Empty DataFrame
  Columns: [eng_Latn, pan_Guru]
  Index: [],
  'child':                                               eng_Latn  \
  568  Children should learn classical music, in my o...   
  569  Is it beneficial for children to study classic...   
  570  Yes. It's quite difficult but classical music ...   
  571  According to studies, listening to live classi...   
  573  Classical music is beneficial to children's de...   
  576  This training will discipline the child's life.\n   
  578                It also soothes children's minds.\n   
  579               Yeah, a calm child is more joyful.\n   
  580  As children learn to listen careful

In [13]:
df_list

{'pan_Guru':                                                   eng_Latn  \
 816              """ You don't know what you're saying."\n   
 817            """ You gotta ask yourself one question."\n   
 818                           """ You have 3 seconds!"""\n   
 819                             """ You have a child?"""\n   
 820      """ You have saved me now three times...I had ...   
 ...                                                    ...   
 6275792                  All other amenities were great.\n   
 6275793   One person died while being taken to hospital.\n   
 6275794                      Aren’t we currently at war?\n   
 6275795            Car falls into canal,5 of family dead\n   
 6275796                                                 \n   
 
                                                   pan_Guru  
 816           """ਤੈਨੂੰ ਪਤਾ ਨਹੀਂ ਕਿ ਤੂੰ ਕੀ ਕਹਿ ਰਹੀਂ ਹੈਂ।"\n  
 817       'ਤੁਹਾਨੂੰ ਆਪਣੇ ਆਪ ਤੋਂ ਇਕ ਸਵਾਲ ਪੁੱਛਣਾ ਚਾਹੀਦਾ ਹੈ.\n  
 818                    """ਤੁਹਾਡੇ ਕੋਲ ਤੀਹ ਸਕ

## Dict of partition of translations

In [4]:
# load file from lang_rel_dict/ory_Orya/ory_Orya_grandmother_bpcc.csv
df_lang_rel = pd.read_csv('lang_rel_dict/ory_Orya/ory_Orya_grandmother_bpcc.csv')

df_lang_rel


,eng_Latn,ory_Orya
0,"""""""Dreams come true, my roommate Pradeep Krish...","ଏହାସହ ନାଗାର୍ଜୁନ ଲେଖିଛନ୍ତି, ' ସ୍ୱପ୍ନ ସତ ହେଲା, ମ..."
1,"""""""I never got a paisa's compensation from gov...","""""""କ୍ଷତିପୂରଣ ବାବଦକୁ ମୁଁ ସରକାରଙ୍କ ଠାରୁ କାଣି କଉଡ..."
2,"""""""I would see the child in only her underwear...","""""""ମୁଁ ଶିଶୁଟିକୁ କେବଳ ଚଡ଼ି ପିନ୍ଧି ଚୁପ୍ଚାପ୍ ତା' ..."
3,"""""""I've become a grandmother,"""" she says, """"bu...","""ସେ କୁହନ୍ତି, """"ମୁଁ ଗୋଟିଏ ଆଈ ହୋଇଯାଇଛି କିନ୍ତୁ ଜଣ..."
4,"""""""My grandmother was the first person that I ...","ସେ କହିଥିଲେ ଯେ, ମୋର ଜେଜେ ହେଉଛନ୍ତି ପ୍ରଥମ ବ୍ୟକ୍ତି..."
...,...,...
353,"He has a grandmother, wife and three children ...","ସେହି ଘରେ ତା ସ୍ତ୍ରୀ କାଞ୍ଚନ, ବୁଢ଼ୀ ମାଆ ଆଉ ପିଲା ତ..."
354,The mother and grandmother have been arrested.\n,ଶାଶୁ-ଶ୍ୱଶୁରଙ୍କୁ ଗିରଫ କରାଯାଇଛି।\n
355,But I am not my Grandmother.\n,କିନ୍ତୁ ମୁଁ ବୋକା ନୁହେଁ।\n
356,"""My grandmother said . . .\n",” ଏନେଇ ମାଆ କହିଲା…\n


In [4]:
possible_odia_grandmother = ['ଜେଜେମା', 'ଆଈ']
# TODO: dictionary of such words language wise with paternal or maternal annoattaion

possible_indic_relations = {
    'grandmother': {
        'ory_Orya': {
            'ଜେଜେମା': {
                'description': 'mother of father',
                'relation': 'paternal',
                'relation_code': 'M', # Male or Female Side of me or my immediate family or BOth or None
                'gender': 'F',


            },
            'ଆଈ': {
                'description': 'mother of mother',
                'relation': 'maternal',
                'relation_code': 'F',
                'gender': 'F'
            }

        },
        'ben_Beng': {
            'ঠাকুরমা': {
                'description': 'mother of father',
                'relation': 'paternal',
                'relation_code': 'M',
                'gender': 'F'
            },
            ' দিদি মা': {
                'description': 'mother of mother',
                'relation': 'maternal',
                'relation_code': 'F',
                'gender': 'F'
            }
           

        },
        'guj_Gujr': {
            'paternal': ['જેઠી'],
            'maternal': ['આઈ']
        },
        'hin_Deva': {
            'paternal': ['दादीमा'],
            'maternal': ['नानी']
        },
        'kan_Knda': {
            'ಅಜ್ಜಿ': {
                'description': 'mother of father or mother',
                'relation': 'both',
                'relation_code': 'B',
                'gender': 'F'
            }
        },
        'mal_Mlym': {
            
            'അമ്മൂമ്മ':
            {
                'description': 'mother of father or mother',
                'relation': 'both',
                'relation_code': 'B',
                'gender': 'F'
            }
        },
        'mar_Deva': {
            'paternal': ['आजी'],
            'maternal': ['आई']
        },
        'tam_Taml': {
            'paternal': ['பாட்டி'],
            'maternal': ['அம்மா']
        },
        'tel_Telu': {
            'paternal': ['నానమ్మ'],
            'maternal': ['అమ్మ']
        },
        'pan_Guru': {
            
            'ਦਾਦੀ':
            {
                'description': 'mother of father',
                'relation': 'paternal',
                'relation_code': 'M',
                'gender': 'F'
            },
            'ਨਾਨੀ':
            {
                'description': 'mother of mother',
                'relation': 'maternal',
                'relation_code': 'F',
                'gender': 'F'
            }
        }

    }
    ,

    'grandfather':{
        'ory_Orya': {
           
           'ଜେଜେ'
        },
        'ben_Beng': {
            'paternal': ['ঠাকুরমা'],
            'maternal': ['দিদা']
        },
        'guj_Gujr': {
            'paternal': ['જેઠી'],
            'maternal': ['આઈ']
        },
        'hin_Deva': {
            'paternal': ['दादीमा'],
            'maternal': ['नानी']
        },
        'kan_Knda': {
            'paternal': ['ಅಜ್ಜಿ'],
            'maternal': ['ತಾಯಿ']
        },
        'mal_Mlym': {
            'paternal': ['പാട്ടിക'],
            'maternal': ['അമ്മമ്മ']
        },
        'mar_Deva': {
            'paternal': ['आजी'],
            'maternal': ['आई']
        },
        'tam_Taml': {
            'paternal': ['பாட்டி'],
            'maternal': ['அம்மா']
        },
        'tel_Telu': {
            'paternal': ['నానమ్మ'],
            'maternal': ['అమ్మ']
        },
        'pan_Guru': {
            'paternal': ['ਦਾਦੀ'],
            'maternal': ['ਨਾਨੀ']
        }

    },
          'uncle': {
        'ory_Orya': {
            'paternal': ['ଜେଜେମା'],
            'maternal': ['ଆଈ']
        },
        'ben_Beng': {
            'paternal': ['ঠাকুরমা'],
            'maternal': ['দিদা']
        },
        'guj_Gujr': {
            'paternal': ['જેઠી'],
            'maternal': ['આઈ']
        },
        'hin_Deva': {
            'paternal': ['दादीमा'],
            'maternal': ['नानी']
        },
        'kan_Knda': {
            'paternal': ['ಅಜ್ಜಿ'],
            'maternal': ['ತಾಯಿ']
        },
        'mal_Mlym': {
            'paternal': ['പാട്ടിക'],
            'maternal': ['അമ്മമ്മ']
        },
        'mar_Deva': {
            'paternal': ['आजी'],
            'maternal': ['आई']
        },
        'tam_Taml': {
            'paternal': ['பாட்டி'],
            'maternal': ['அம்மா']
        },
        'tel_Telu': {
            'paternal': ['నానమ్మ'],
            'maternal': ['అమ్మ']
        },
        'pan_Guru': {
            'paternal': ['ਦਾਦੀ'],
            'maternal': ['ਨਾਨੀ']
        }

    },
            'aunt': {
        'ory_Orya': {
            'paternal': ['ଜେଜେମା'],
            'maternal': ['ଆଈ']
        },
        'ben_Beng': {
            'paternal': ['ঠাকুরমা'],
            'maternal': ['দিদা']
        },
        'guj_Gujr': {
            'paternal': ['જેઠી'],
            'maternal': ['આઈ']
        },
        'hin_Deva': {
            'paternal': ['दादीमा'],
            'maternal': ['नानी']
        },
        'kan_Knda': {
            'paternal': ['ಅಜ್ಜಿ'],
            'maternal': ['ತಾಯಿ']
        },
        'mal_Mlym': {
            'paternal': ['പാട്ടിക'],
            'maternal': ['അമ്മമ്മ']
        },
        'mar_Deva': {
            'paternal': ['आजी'],
            'maternal': ['आई']
        },
        'tam_Taml': {
            'paternal': ['பாட்டி'],
            'maternal': ['அம்மா']
        },
        'tel_Telu': {
            'paternal': ['నానమ్మ'],
            'maternal': ['అమ్మ']
        },
        'pan_Guru': {
            'paternal': ['ਦਾਦੀ'],
            'maternal': ['ਨਾਨੀ']
        }

    },
      'brother-in-law': {
        'ory_Orya': {
            'paternal': ['ଜେଜେମା'],
            'maternal': ['ଆଈ']
        },
        'ben_Beng': {
            'paternal': ['ঠাকুরমা'],
            'maternal': ['দিদা']
        },
        'guj_Gujr': {
            'paternal': ['જેઠી'],
            'maternal': ['આઈ']
        },
        'hin_Deva': {
            'paternal': ['दादीमा'],
            'maternal': ['नानी']
        },
        'kan_Knda': {
            'paternal': ['ಅಜ್ಜಿ'],
            'maternal': ['ತಾಯಿ']
        },
        'mal_Mlym': {
            'paternal': ['പാട്ടിക'],
            'maternal': ['അമ്മമ്മ']
        },
        'mar_Deva': {
            'paternal': ['आजी'],
            'maternal': ['आई']
        },
        'tam_Taml': {
            'paternal': ['பாட்டி'],
            'maternal': ['அம்மா']
        },
        'tel_Telu': {
            'paternal': ['నానమ్మ'],
            'maternal': ['అమ్మ']
        },
        'pan_Guru': {
            'paternal': ['ਦਾਦੀ'],
            'maternal': ['ਨਾਨੀ']
        }

    },
    'sister-in-law':{
        'ory_Orya': {
            'paternal': ['ଜେଜେମା'],
            'maternal': ['ଆଈ']
        },
        'ben_Beng': {
            'paternal': ['ঠাকুরমা'],
            'maternal': ['দিদা']
        },
        'guj_Gujr': {
            'paternal': ['જેઠી'],
            'maternal': ['આઈ']
        },
        'hin_Deva': {
            'paternal': ['दादीमा'],
            'maternal': ['नानी']
        },
        'kan_Knda': {
            'paternal': ['ಅಜ್ಜಿ'],
            'maternal': ['ತಾಯಿ']
        },
        'mal_Mlym': {
            'paternal': ['പാട്ടിക'],
            'maternal': ['അമ്മമ്മ']
        },
        'mar_Deva': {
            'paternal': ['आजी'],
            'maternal': ['आई']
        },
        'tam_Taml': {
            'paternal': ['பாட்டி'],
            'maternal': ['அம்மா']
        },
        'tel_Telu': {
            'paternal': ['నానమ్మ'],
            'maternal': ['అమ్మ']
        },
        'pan_Guru': {
            'paternal': ['ਦਾਦੀ'],
            'maternal': ['ਨਾਨੀ']
        }

    }   
   , 'cousin': {
        'ory_Orya': {
            'paternal': ['ଜେଜେମା'],
            'maternal': ['ଆଈ']
        },
        'ben_Beng': {
            'paternal': ['ঠাকুরমা'],
            'maternal': ['দিদা']
        },
        'guj_Gujr': {
            'paternal': ['જેઠી'],
            'maternal': ['આઈ']
        },
        'hin_Deva': {
            'paternal': ['दादीमा'],
            'maternal': ['नानी']
        },
        'kan_Knda': {
            'paternal': ['ಅಜ್ಜಿ'],
            'maternal': ['ತಾಯಿ']
        },
        'mal_Mlym': {
            'paternal': ['പാട്ടിക'],
            'maternal': ['അമ്മമ്മ']
        },
        'mar_Deva': {
            'paternal': ['आजी'],
            'maternal': ['आई']
        },
        'tam_Taml': {
            'paternal': ['பாட்டி'],
            'maternal': ['அம்மா']
        },
        'tel_Telu': {
            'paternal': ['నానమ్మ'],
            'maternal': ['అమ్మ']
        },
        'pan_Guru': {
            'paternal': ['ਦਾਦੀ'],
            'maternal': ['ਨਾਨੀ']
        }

    }
,'child':      {
        'ory_Orya': {
            'paternal': ['ଜେଜେମା'],
            'maternal': ['ଆଈ']
        },
        'ben_Beng': {
            'paternal': ['ঠাকুরমা'],
            'maternal': ['দিদা']
        },
        'guj_Gujr': {
            'paternal': ['જેઠી'],
            'maternal': ['આઈ']
        },
        'hin_Deva': {
            'paternal': ['दादीमा'],
            'maternal': ['नानी']
        },
        'kan_Knda': {
            'paternal': ['ಅಜ್ಜಿ'],
            'maternal': ['ತಾಯಿ']
        },
        'mal_Mlym': {
            'paternal': ['പാട്ടിക'],
            'maternal': ['അമ്മമ്മ']
        },
        'mar_Deva': {
            'paternal': ['आजी'],
            'maternal': ['आई']
        },
        'tam_Taml': {
            'paternal': ['பாட்டி'],
            'maternal': ['அம்மா']
        },
        'tel_Telu': {
            'paternal': ['నానమ్మ'],
            'maternal': ['అమ్మ']
        },
        'pan_Guru': {
            'paternal': ['ਦਾਦੀ'],
            'maternal': ['ਨਾਨੀ']
        }

    }
,'nephew'    : {
        'ory_Orya': {
            'paternal': ['ଜେଜେମା'],
            'maternal': ['ଆଈ']
        },
        'ben_Beng': {
            'paternal': ['ঠাকুরমা'],
            'maternal': ['দিদা']
        },
        'guj_Gujr': {
            'paternal': ['જેઠી'],
            'maternal': ['આઈ']
        },
        'hin_Deva': {
            'paternal': ['दादीमा'],
            'maternal': ['नानी']
        },
        'kan_Knda': {
            'paternal': ['ಅಜ್ಜಿ'],
            'maternal': ['ತಾಯಿ']
        },
        'mal_Mlym': {
            'paternal': ['പാട്ടിക'],
            'maternal': ['അമ്മമ്മ']
        },
        'mar_Deva': {
            'paternal': ['आजी'],
            'maternal': ['आई']
        },
        'tam_Taml': {
            'paternal': ['பாட்டி'],
            'maternal': ['அம்மா']
        },
        'tel_Telu': {
            'paternal': ['నానమ్మ'],
            'maternal': ['అమ్మ']
        },
        'pan_Guru': {
            'paternal': ['ਦਾਦੀ'],
            'maternal': ['ਨਾਨੀ']
        }

    }
,'niece':
   {
        'ory_Orya': {
            'paternal': ['ଜେଜେମା'],
            'maternal': ['ଆଈ']
        },
        'ben_Beng': {
            'paternal': ['ঠাকুরমা'],
            'maternal': ['দিদা']
        },
        'guj_Gujr': {
            'paternal': ['જેઠી'],
            'maternal': ['આઈ']
        },
        'hin_Deva': {
            'paternal': ['दादीमा'],
            'maternal': ['नानी']
        },
        'kan_Knda': {
            'paternal': ['ಅಜ್ಜಿ'],
            'maternal': ['ತಾಯಿ']
        },
        'mal_Mlym': {
            'paternal': ['പാട്ടിക'],
            'maternal': ['അമ്മമ്മ']
        },
        'mar_Deva': {
            'paternal': ['आजी'],
            'maternal': ['आई']
        },
        'tam_Taml': {
            'paternal': ['பாட்டி'],
            'maternal': ['அம்மா']
        },
        'tel_Telu': {
            'paternal': ['నానమ్మ'],
            'maternal': ['అమ్మ']
        },
        'pan_Guru': {
            'paternal': ['ਦਾਦੀ'],
            'maternal': ['ਨਾਨੀ']
        }

    }

}


In [ ]:
# # cross the no.of langs in each word

# for word in ambiguos:
#     for lang in df_dict:
#         if word in df_dict[lang]:
#             print(lang, word, df_dict[lang][word].shape)

In [7]:
# make a dictionary odia_grandmother_dict with key as the word from possible_odia_grandmother and value as the 
# dataframe containing that row from df_odia_grandmother.
# if any sentence is not having any possible word from possible_odia_grandmother, then add that sentence to the
# 'others' key in the odia_grandmother_dict

result_dict = {}
for word in ambiguos:
    result_dict[word] = {}
    for lang in df_dict:
        lang_dict = {}
        for indic_word in possible_indic_relations[word][lang]['paternal']:
            lang_dict[indic_word] = df_lang_rel[df_lang_rel[lang].str.contains(indic_word, case=False, )]
            print(indic_word, lang_dict[indic_word].shape)
        
        lang_dict['others'] = df_lang_rel[~df_lang_rel[lang].str.contains('|'.join(possible_odia_grandmother), case=False, )]
        print('others', lang_dict['others'].shape)
        result_dict[word][lang] = lang_dict

lang_dict

ଜେଜେମା (121, 2)
ଆଈ (6, 2)
others (231, 2)


{'ଜେଜେମା':                                               eng_Latn  \
 0    """Dreams come true, my roommate Pradeep Krish...   
 1    """I never got a paisa's compensation from gov...   
 2    """I would see the child in only her underwear...   
 5        """My grandmother worked outside the home."\n   
 6                      """Now you see, grandmother!"\n   
 ..                                                 ...   
 326                            the grandmother died.\n   
 327                          the grandmother's house\n   
 329                   where my grandmother was born.\n   
 347  New Delhi: Congress chief Rahul Gandhi paid tr...   
 351  At home, there is my grandmother, grandfather,...   
 
                                               ory_Orya  
 0    ଏହାସହ ନାଗାର୍ଜୁନ ଲେଖିଛନ୍ତି, ' ସ୍ୱପ୍ନ ସତ ହେଲା, ମ...  
 1    """କ୍ଷତିପୂରଣ ବାବଦକୁ ମୁଁ ସରକାରଙ୍କ ଠାରୁ କାଣି କଉଡ...  
 2    """ମୁଁ ଶିଶୁଟିକୁ କେବଳ ଚଡ଼ି ପିନ୍ଧି ଚୁପ୍ଚାପ୍ ତା' ...  
 5                     ଜେଜେମା' ଘର ବାହାରେ କାମ କରୁ

# defining relations

In [18]:
def contains_reln(sent, list_relations, is_eng=False):
  # print(sent, list_relations[0])
  # for reln in list_relations:
  #       if reln in sent:
  #           # print(sent, reln)
  #           return True
  # return False
  # check if any of the relations in list_relations are present in the sentence sent. do check with lower case if lowercase is allowed
  if is_eng:
      for reln in list_relations:
        if reln.lower() in sent.lower():
            # print(sent, reln)
            return True
  else:
      return any(reln in sent for reln in list_relations)





In [19]:


relations_odia = ['ବାପା',  # Father
                  'ପିତା', 'ମାତା', 'ବୋଉ'
'ନାନା',  # Nānā
    # 'ମା\'',  # Mother
    'ଭାଇ',  # Brother
    'ଭଉଣୀ',  # Sister
    'ପୁଅ',  # Son
    'ଝିଅ',  # Daughter
    'ଦିଦି',  # Didi
    'ଜେଜେମା',  # Jejemā
    'ଜେଜେବାପା',  # Jejebāpā
    'ନାତି',  # Nāti
    'ନାତୁଣୀ',  # Nātuṇī
    'ଅଣନାତି',  # Aṇanāti
    'ଅଣନାତୁଣୀ',  # Aṇanātuṇī
    'ପଣନାତି',  # Paṇanāti
    'ପଣନାତୁଣୀ',  # Paṇanātuṇī
    'ଶଳା',  # Śālā
    'ଶାଳୀ',  # Śālī
    'ଭାଉଜ',  # Bhāuja
    'ନଣନ୍ଦ',  # Nanannda
    'ଭିନେଇ',  # Bhinēi
    'ଦିଆର',  # Diāra
    # 'ଜା\'',  # Jā
    'ଭାଇବୋହୁ',  # Bhāibohu
    'ନଣନ୍ଦେଇ',  # Nanandei
   'ଦାଦା',  # Dādā
    'ଖୁଡ଼ି',  # Khudi
    'ବଡ଼ବାପା',  # Baḍabāpā
    'ବଡ଼ମା',  # Baḍamā
    'ମାମୁ',  # Māmu
    'ମାଇଁ',  # Māiṁ
   'ମାଉସୀ',
'ମଉସା',
    'ପିସା',  # Pisā
    'ପିସୀ',  # Pisī
    'ଝିଆରୀ',  # Jhīārī
    'ପୁତୁରା',  # Puturā
    'ଭଣଜା',  # Bhāṇajā
      # Bhāṇajī
      'ଜ୍ୱାଇଁ',  # Jwāiṁ
    'ଶାଶୁ',  # Śāśu
    'ଶ୍ୱଶୁର'  # Śāśurā
    'ବୋହୁ',  # Bohu
    'ସମୁଧି',  # Samudhi
    'ସମୁଧୁନୀ',  # Samudhuni
    'ଜେଜେଶାଶୁ',  # Jejeśāśu
    'ସ୍ୱାମୀ',  # Swāmī
    'ସ୍ତ୍ରୀ',  # Strī
    'ବୋଉଲା',  # Boula
    'ସଙ୍ଗାତ' ,
'ଭାଣିଜୀ',
'ଦେଢ଼ଶୁର',
'ଦେଢଶଶୁର  ','ଦେଢ଼ଶାସୁ ','ସଢ଼ୁ','ସାବତ', 'ଅଜା', 'ଆଈ']

relations_hindi =['पिता', 'माँ', 'भाई', 'बहन', 'बेटा', 'बेटी', 'दीदी', 'दादी माँ', 'दादा जी', 'दादा',
                  'नाना', 'नानी', 'चाचा', 'चाची', 'मामा', 'मामी', 'भतीजा', 'भतीजी', 'भांजा', 'भांजी',
                  'जीजा', 'जीजी', 'देवर', 'देवरानी', 'जेठ', 'जेठानी', 'सास', 'सासु', 'ननद', 
                  'भाभी',  'बहू', 'पोता','पोती', 'नातिन', 'साला','साली', 'परपोती ', 'परपोता ',  'मौसा ',
                   'मौसी ',  'भतीजे ',  'दामाद ', 'सास','ससुर ', 'बहनोई ', 'पति ', 'पत्नी ', 'सौतेला', 'सौतेली',
                   'ननदोई', 'काका', 'काकी', 'बुआ', 'फूफा',  'समधी', 'समधन', 
                   
                   ]

print("Lengths:", len(relations_odia), len(relations_hindi))

Lengths: 56 56


In [13]:
relations_english = ['father',  'mother', 'brother', 'sister', 'son',
                     'daughter', 'grandfather', 'grandmother', 'granddaughter', 'grandson',
                     'husband', 'wife', 'uncle', 'aunt', 'niece', 'nephew',
                    'fiancé', 'fiancée', 'daughter-in-law', 'son-in-law',
                     'sister-in-law', 'brother-in-law']

# Translate using models


In [4]:

import os
import pandas as pd

test_csv_folder = 'test_csv__'

In [5]:
lang_code_map = {
    'eng_Latn': 'en',
    'hin_Deva': 'hi',
    'guj_Gujr': 'gu',
    'kan_Knda': 'kn',
    'mal_Mlym': 'ml',
    'mar_Deva': 'mr',
    'tam_Taml': 'ta',
    'tel_Telu': 'te',
    'pan_Guru': 'pa',
    'ben_Beng': 'bn',
    'ory_Orya': 'or'
}


## IndicTrans2

In [9]:
import torch
from transformers import AutoModelForSeq2SeqLM, BitsAndBytesConfig
# from IndicTransTokenizer import IndicProcessor, IndicTransTokenizer
from IndicTransToolkit import IndicProcessor
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

BATCH_SIZE = 16 # edited from 4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
quantization = None
print(DEVICE)

cuda


In [10]:
def initialize_model_and_tokenizer(ckpt_dir, quantization):
    if quantization == "4-bit":
        qconfig = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
        )
    elif quantization == "8-bit":
        qconfig = BitsAndBytesConfig(
            load_in_8bit=True,
            bnb_8bit_use_double_quant=True,
            bnb_8bit_compute_dtype=torch.bfloat16,
        )
    else:
        qconfig = None

    tokenizer = AutoTokenizer.from_pretrained(ckpt_dir, trust_remote_code=True)
    model = AutoModelForSeq2SeqLM.from_pretrained(
        ckpt_dir,
        trust_remote_code=True,
        low_cpu_mem_usage=True,
        quantization_config=qconfig,
    )

    if qconfig == None:
        model = model.to(DEVICE)
        if DEVICE == "cuda":
            model.half()

    model.eval()

    return tokenizer, model


def batch_translate(input_sentences, src_lang, tgt_lang, model, tokenizer, ip):
    translations = []


    from tqdm import tqdm
    for i in tqdm(range(0, len(input_sentences), BATCH_SIZE)):

    # for i in range(0, len(input_sentences), BATCH_SIZE):
        batch = input_sentences[i : i + BATCH_SIZE]

        # Preprocess the batch and extract entity mappings
        batch = ip.preprocess_batch(batch, src_lang=src_lang, tgt_lang=tgt_lang)

        # Tokenize the batch and generate input encodings
        inputs = tokenizer(
            batch,
            truncation=True,
            padding="longest",
            return_tensors="pt",
            return_attention_mask=True,
        ).to(DEVICE)

        # Generate translations using the model
        with torch.no_grad():
            generated_tokens = model.generate(
                **inputs,
                use_cache=True,
                min_length=0,
                max_length=256,
                num_beams=5,
                num_return_sequences=1, # TODO temp
            )

        # Decode the generated tokens into text

        with tokenizer.as_target_tokenizer():
            generated_tokens = tokenizer.batch_decode(
                generated_tokens.detach().cpu().tolist(),
                skip_special_tokens=True,
                clean_up_tokenization_spaces=True,
            )

        # Postprocess the translations, including entity replacement
        translations += ip.postprocess_batch(generated_tokens, lang=tgt_lang)

        del inputs
        torch.cuda.empty_cache()

    return translations

In [11]:
en_indic_ckpt_dir = "ai4bharat/indictrans2-en-indic-1B"  # ai4bharat/indictrans2-en-indic-dist-200M
en_indic_tokenizer, en_indic_model = initialize_model_and_tokenizer(en_indic_ckpt_dir,  quantization)

ip_en_ind = IndicProcessor(inference=True)

In [ ]:
# # Initialize model and tokenizer for Indic to English
# indic_en_ckpt_dir = "ai4bharat/indictrans2-indic-en-1B"
# indic_en_tokenizer, indic_en_model = initialize_model_and_tokenizer(indic_en_ckpt_dir, "indic-en", quantization)
# ip_ind_en = IndicProcessor(inference=True)

In [7]:
# CODE for INDIC to INDIC MODEL
# indic_indic_ckpt_dir = "ai4bharat/indictrans2-indic-indic-1B"  # ai4bharat/indictrans2-indic-indic-dist-320M
# indic_indic_tokenizer, indic_indic_model = initialize_model_and_tokenizer(indic_indic_ckpt_dir, quantization)

# ip_indic_indic = IndicProcessor(inference=True)

In [17]:
src_lang, tgt_lang =  "eng_Latn", "hin_Deva",
hi_translations = batch_translate(sents, src_lang, tgt_lang, en_indic_model, en_indic_tokenizer, ip_en_ind)


  0%|          | 0/282 [00:00<?, ?it/s]/home/sofia/anaconda3/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
100%|██████████| 282/282 [20:55<00:00,  4.45s/it]


## Sarvam AI

## Microsoft Phi

In [10]:
!pip install requests uuid

  Preparing metadata (setup.py) ... done
  Created wheel for uuid: filename=uuid-1.30-py3-none-any.whl size=6478 sha256=65ce5cdcec8beb6e4148714b8f5e944577d694123c06c496289f5079eb76bd80
  Stored in directory: /home/sofia/.cache/pip/wheels/05/d7/b4/4795d29c6decfffbf64c63e58b6c8b8bbfd4751488617dcd7a
Successfully built uuid


In [11]:
import requests, uuid, json

# Add your key and endpoint
key = "f1d5da8e40cc4634b631f7ec807676e1"
endpoint = "https://api.cognitive.microsofttranslator.com"

# location, also known as region.
# required if you're using a multi-service or regional (not global) resource. It can be found in the Azure portal on the Keys and Endpoint page.
location = "centralindia"

path = '/translate'
constructed_url = endpoint + path

In [14]:


headers = {
    'Ocp-Apim-Subscription-Key': key,
    # location required if you're using a multi-service or regional (not global) resource.
    'Ocp-Apim-Subscription-Region': location,
    'Content-type': 'application/json',
    'X-ClientTraceId': str(uuid.uuid4())
}

In [17]:
  
for folder in os.listdir(test_csv_folder):
    print(folder)
    for file in os.listdir(os.path.join(test_csv_folder, folder)):
        print(file)
        df = pd.read_csv(test_csv_folder+'/'+folder+'/'+file)
        # print(lang_code_map[folder], df.head())
        body=[]
        for text in df['eng_Latn']:
            text_object = {'text': text}
            body.append(text_object)

        params = {
            'api-version': '3.0',
            'from': 'en',
            'to': lang_code_map[folder]
        }

          # if df is not empty and df[eng_Latn] is not empty:
        if not df.empty and not df['eng_Latn'].empty:
            request = requests.post(constructed_url, params=params, headers=headers, json=body)
            response = request.json()
            # read from the translated csv files if exsists and add the newly translated sents to last column
            # if not exists, then create a new csv file with the eng_Latn, folder and  translated sents

            if os.path.exists(os.path.join('translated_csv', folder, file)):
                df_translated = pd.read_csv(os.path.join('translated_csv', folder, file))
                for i in range(len(response)):
                    df_translated.loc[i, 'transl_ms_phi'] = response[i]['translations'][0]['text']
                df_translated.to_csv(os.path.join('translated_csv', folder, file), index=False)
            else:
                df['transl_ms_phi'] = [response[i]['translations'][0]['text'] for i in range(len(response))]
                df.to_csv(os.path.join('translated_csv', folder, file), index=False)



tam_Taml
tam_Taml_cousins'_IN22_testset.csv
tam_Taml_child_IN22_testset.csv
tam_Taml_nieces'_IN22_testset.csv
tam_Taml_grandfather's_IN22_testset.csv
tam_Taml_nephew_IN22_testset.csv
tam_Taml_sister in law's_IN22_testset.csv
tam_Taml_grandmothers'_IN22_testset.csv
tam_Taml_cousin_IN22_testset.csv
tam_Taml_sisters in law's_IN22_testset.csv
tam_Taml_brother-in-law's_IN22_testset.csv
tam_Taml_aunts_IN22_testset.csv
tam_Taml_uncle's_IN22_testset.csv
tam_Taml_grandmothers_IN22_testset.csv
tam_Taml_cousins_IN22_testset.csv
tam_Taml_niece_IN22_testset.csv
tam_Taml_nephews_IN22_testset.csv
tam_Taml_uncles_IN22_testset.csv
tam_Taml_brothers-in-law's_IN22_testset.csv
tam_Taml_children_IN22_testset.csv
tam_Taml_aunt's_IN22_testset.csv
tam_Taml_grandfathers'_IN22_testset.csv
tam_Taml_brothers-in-law_IN22_testset.csv
tam_Taml_niece's_IN22_testset.csv
tam_Taml_uncles'_IN22_testset.csv
tam_Taml_aunts'_IN22_testset.csv
tam_Taml_sister-in-law's_IN22_testset.csv
tam_Taml_sisters-in-law's_IN22_testset.cs

## Google Translate API


In [1]:
# pip install googletrans
!pip install googletrans


In [3]:
from googletrans import Translator


translator = Translator(service_urls=['translate.googleapis.com'])
translator

In [6]:
translator.translate('hello', src='en', dest='hi').text

'नमस्ते'

In [18]:
# for each folder of language in test_csv__ folder, translate the csv file's 1st column from English to that language
# and save the translation as last column in the same csv file with name as 'transl_google_trans'

for folder in os.listdir(test_csv_folder):
    print(folder)
    for file in os.listdir(os.path.join(test_csv_folder, folder)):
        print(file)
        df = pd.read_csv(test_csv_folder+'/'+folder+'/'+file)
        print(lang_code_map[folder], df.head())
          # if df is not empty and df[eng_Latn] is not empty:
        if not df.empty and not df['eng_Latn'].empty:
          transls= df['eng_Latn'].apply(lambda x: translator.translate(x, src='en', dest=lang_code_map[folder]).text)

          if os.path.exists(os.path.join('translated_csv', folder, file)):
              df_translated = pd.read_csv(os.path.join('translated_csv', folder, file))

              df_translated.loc[i, 'transl_google_trans'] = transls
              df_translated.to_csv(os.path.join('translated_csv', folder, file), index=False)
          else:
              df['transl_google_trans'] = transls
              df.to_csv(os.path.join('translated_csv', folder, file), index=False)

            



tam_Taml
tam_Taml_cousins'_IN22_testset.csv
ta Empty DataFrame
Columns: [eng_Latn, tam_Taml]
Index: []
tam_Taml_child_IN22_testset.csv
ta                                             eng_Latn  \
0  Children should learn classical music, in my o...   
1  Is it beneficial for children to study classic...   
2  Yes. It's quite difficult but classical music ...   
3  According to studies, listening to live classi...   
4  Classical music is beneficial to children's de...   

                                            tam_Taml  
0  என்னைப் பொறுத்தவரை குழந்தைகள் கிளாசிக்கல் மியூ...  
1  கிளாசிக்கல் மியூசிக்கைக் கற்றுக்கொள்வது குழந்த...  
2  ஆமாம், கிளாசிக்கல் மியூசிக் கற்றுக்கொள்ள கடினம...  
3  ஆய்வுகளின்படி, கிளாசிக்கல் மியூசிக் கேட்பதால் ...  
4  குழந்தையின் வளர்ச்சிக்கு கிளாசிக்கல் மியூசிக் ...  


ValueError: Incompatible indexer with Series

# Conclusion